In [3]:
pos_words = ['jumped', 'hike', 'good','rise','rose', 'growth', 'falling interest rates',\
             'bull','truce','bullish','bull','bullish','bulls','bullish','bull','bullish', 'great',\
             'optimistic', 'rally', 'surge', 'loved', 'dividend increases', 'benifit'\
             'soared', 'growth' 'loss','buy', 'higher','break resistance','gain','better', 'gains', 'outperform',\
             'well', 'solid', 'regret not buying','keep buying', 'best', 'strong'\
             'hot', 'love','hottest', 'best', 'beat analyst expectations','up', 'frontrunner', 'new funding',
             'fresh funding', 'acquire', 'high','higher','highest','high'\
            ]
neg_words = ['bearish', 'bears','bad','poor','expansion','slow', 'bear','bearish', 'bears', 'bear','bearish', 'bears', 'bear',\
             'bear','bearish','underperform', 'rising interest rates', 'lower','declared unsafe',\
             'caught','cheap for a reason','caution','cautions','criticized','decline', 'toll' \
             'slumped','market drop','regret not selling','brace','pressure', 'fell','terrible','awful', 'profit','worry','fell below support',\
             'bearish', 'miss','shock','fell short of analyst expectations','fizzle','pullback'\
             'sell', 'losses','fail','not buying','muted', 'slide','warn','worst', 'rattle','stale',\
             'plummet', 'down', 'low', 'disappointed', 'weak', 'slow', 'growth', 'stalled growth',\
             'worry', 'low', 'warning','struggle','retreat', 'mute', 'fear', 'downside']

In [4]:
print(len(pos_words), len(neg_words))

50 67


In [5]:
with open('20k.txt','r') as f:
    common_words = f.read()
common_words = common_words.split('\n')[:20_000]
hyper_common_words, common_words = common_words[:100],common_words[100:]

In [6]:
finance_words = """Actively Managed Funds
Fibonacci retracement
Pennant
sentiment
flag
wedge
Bollinger Bands
sma
simple moving average
donald trump
boss
corporate
bussiness
politics
president
alpha
beta
price
bond
blue chip
investing
Business Inventories
buy side 
asset
capital
Capital Expenditure
Collateralized Debt Obligation
Common Stock
Consumer Price Index
Convertible Securities
Convexity
Corporate Bonds
Covered Call
Credit Default Swap
Credit Risk
Credit Spread
Crude Oil
bitcoin
gold
silver
yen
dollar
earning
Dow Jones Industrial Average
dow
Dogs of the Dow
EBITDA
Economic Indicators
Federal Reserve
Federal Open Market Committee
Federal Funds Rate
Float
Fundamentals
Gross Domestic Product
Growth Funds
Head and Shoulders
Hedge Fund
Housing Starts
International Monetary Fund
Initial Jobless Claims
International Funds
Interest Rate
cut
take
Intrinsic Value
Investment Bank
Moving Average
Municipal Bonds
Put
Call
earning call
Passively Managed Funds
PEG Ratio
Preferred Stock
Price Earnings
option
S&P 500
Short Selling
Spread
Spread Product
Standard Deviation
Support
Swaps
Treasury Bills
Treasury Futures
Treasury Notes
Treasury Securities
Treasury Bond
Value Stock
VIX
Volatility
When Issued Trading
Whisper Number
Yield Curve""".split('\n')

In [7]:
import pandas as pd
import numpy as np
with open('companies.csv','r') as f:
    companies = pd.read_csv(f).as_matrix()
companies[np.random.randint(0,504)]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


array(['ANTM', 'Anthem Inc.', 'Health Care'], dtype=object)

In [9]:
#pos_words
#neg_words
#common_words
#finance_words
#companies
def gen_sentence(company):
    opinion = np.random.choice([0,1,2], p=[0.4,0.2,0.4])
    if opinion==1:
        corpus_set = [common_words, hyper_common_words, finance_words, company]
        p_set = [0.4,0.3,0.2,0.1]
    else:
        sentiment_set = [neg_words,'', pos_words][opinion]
        corpus_set = [sentiment_set, common_words, hyper_common_words, finance_words, company]
        p_set = [0.25,0.25,0.25,0.2,0.05]
    sen = np.random.permutation([company[0], company[1], company[2]]).tolist()
    for i in range(0, np.random.randint(10,80)):
        word_set = np.random.choice(corpus_set, p=p_set)
        if  word_set is common_words:
            common_word_distrobution = \
            np.arange(len(common_words), 0,-1)/sum(np.arange(len(common_words), 0,-1))
            word = np.random.choice(common_words, p=common_word_distrobution/sum(common_word_distrobution))
        else:
            word = np.random.choice(word_set)
        sen.insert(np.random.randint(0,len(sen)), word)
    return ' '.join(sen), opinion
gen_sentence(companies[22])

('sma because online Business Inventories view ALLE confusion at underperform Call Standard Deviation are built home seventeen fingering information us Consumer Price Index site fizzle Industrials Credit Spread initial use Bollinger Bands proc bearish been Passively Managed Funds stale poor bear out blue chip Passively Managed Funds link brace criticized decline also view information warn Value Stock Support bussiness bear oldies Common Stock S&P 500 S&P 500 ten Capital Expenditure Allegion',
 0)

In [10]:
import os
data_path = '../../dependencies/labeled_data/meta'
import pickle 
with open(data_path, 'rb') as f:
    data_raw = pickle.load(f)
X,y = np.array(data_raw['X']), np.array(data_raw['y'])

In [11]:
class filter_to_fin():
    def __init__(self, model_path:str = '../models/fin_not_fin_v2.model', inverse_recall_rate:float = 0.4):
        self.model_path = model_path
        self.inverse_recall_rate = inverse_recall_rate
    def transform(self, X, y=None):
        with open(self.model_path, 'rb') as f:
            m_t = pickle.load(f)
        #we want a pretty lose recall here, as most of the things coming will be financial news
        indexes = m_t['model'].predict_proba(m_t['transformer'].transform(X))[:,1]>self.inverse_recall_rate
        return X[indexes], y[indexes]
from sklearn.pipeline import TransformerMixin, Pipeline
from sklearn.base import  BaseEstimator
import urlextract
import re
class Cleaner(BaseEstimator, TransformerMixin):
    def __init__(self, include_subj=True, replace_html=True, remove_punctuation=True, replace_urls=True, replace_numbers=True):
        self.include_subj = include_subj; self.replace_html = replace_html
        self.remove_punctuation = remove_punctuation; self.replace_urls = replace_urls;
        self.replace_numbers = replace_numbers  
    def fit(self, X, y=None):
        return self
    def __html_to_plain_text__(self, html: str) -> str:
        from bs4 import BeautifulSoup
        return BeautifulSoup(html, 'html.parser').get_text()
    def transform(self, X, y=None):
        X_transformed = []
        for article in X:
            text = " ".join(article) if self.include_subj else " ".join(article[1:])
            if self.replace_html:
                text = self.__html_to_plain_text__(text)
            if self.replace_urls:
                url_extractor = urlextract.URLExtract() 
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', text)
            if self.remove_punctuation:
                text = text.replace("\'", "").replace("’", "") #Because we dont want these to be replaced by spaces
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            X_transformed.append(text)
        return X_transformed
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
class CountVectorizerWithStemming(CountVectorizer):
    def build_analyzer(self):
        lemm = WordNetLemmatizer()
        analyzer = super(CountVectorizerWithStemming, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [12]:
#using the fin_not_fin model to filter out non-financial news
X_filtered, y_filtered = filter_to_fin(inverse_recall_rate=0.1).transform(X,y)

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.18 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.18 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.18 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  User

In [13]:
len(X_filtered), len(y_filtered)

(1160, 1160)

In [14]:
gen_set_x = []
gen_set_y = []

In [15]:
for _ in range(5_000):
    sentence, sentiment = gen_sentence(companies[np.random.randint(0,len(companies))])
    gen_set_x.append(['artifical gen','','',sentence])
    gen_set_y.append(sentiment)

In [16]:
print(len(X_filtered), len(gen_set_x))
data_raw = {'X':X_filtered.tolist()+gen_set_x,'y':y_filtered.tolist()+gen_set_y}

1160 5000


In [17]:
len(data_raw['y']), len(data_raw['X'])

(6160, 6160)

In [18]:
with open('train_extended', 'wb') as fp:
            pickle.dump(data_raw, fp)

In [411]:
"Abdsd "*80

'Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd Abdsd '

In [435]:
from nltk.stem import *
stemmer = PorterStemmer()
stemmer.stem('downside')

'downsid'

In [19]:
os.getcwd()

'/Users/DavidGold/Desktop/SchoolWork/GitHubProjects/summer2k19/datascience/NewsClassifier/NewsClassifierWorkbooks/model_notebooks'